<a href="https://colab.research.google.com/github/sethgis/dash_front_end/blob/main/NBR_LANDSAT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installs geemap package
import subprocess
 
try:
    import geemap
    import geopandas as gpd
    import numpy as np
    import folium
    import rasterio
    import tensorflow
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geopandas'])
    subprocess.check_call(["python", '-m', 'pip', 'install', 'folium'])
    subprocess.check_call(["python", '-m', 'pip', 'install', "rasterio"])
    subprocess.check_call(["python", '-m', 'pip', 'install', "tensorflow"])

    
 
# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap
 
# Authenticates and initializes Earth Engine
import ee
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import geopandas as gpd
from shapely.geometry import Point, Polygon
from geemap import geojson_to_ee, ee_to_geojson
from geopandas import GeoSeries
import json
import pandas as pd
%matplotlib inline 
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

geemap package not installed. Installing ...
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=fWd3IZAnC5O51P9I5oLOWacXyFF20bbUruvld94K6sM&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWgiztFqnM2oa0AsdQ1GbCVbHcx2vN_mFp_PbG4VJGOeydks5KPhfXA

Successfully saved authorization token.


In [ ]:
AOI = ee.FeatureCollection('users/snyawacha/ngorongoro_forest_zone')
# AOI.getInfo()

In [ ]:
START_DATE = '2020-01-01'
END_DATE = '2020-03-29'
START_DATE2 = '2020-03-16'
END_DATE2 = '2020-06-29'
CLOUD_FILTER = 60
CLD_PRB_THRESH = 40
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 2
BUFFER = 100
base_period = ['2021-06-01','2021-06-28'] # Image before fire occurence date 


comparissonPeriod = ['2021-07-01', '2021-07-30']

In [ ]:
Start_date = ['2021-06-01','2021-06-28']
End_date =['2021-07-01','2021-07-30']

In [ ]:
# Use Landsat 8 surface reflectance data for predictors.
def Nbr_product(AOI,Start_date, End_date):
  L8SR = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
      .filterBounds(ee.FeatureCollection(AOI)) \
      .filterMetadata('CLOUD_COVER', 'less_than', 60) \
      .sort("CLOUD_COVER")

  # Use these bands for prediction.
  BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7']

  # Cloud masking function.
  def maskL8sr(image):
    cloudShadowBitMask = ee.Number(2).pow(3).int()
    cloudsBitMask = ee.Number(2).pow(5).int()
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(
      qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask).select(BANDS).divide(10000)

  # Acquiring a cloud masked image of Landsat.
  Analysis = L8SR.filterDate(Start_date[0], Start_date[1]).map(maskL8sr)
  Reference = L8SR.filterDate(End_date[0], End_date[1]).map(maskL8sr)

  # Self refernceing the NBR for comparability 
  def NBR(collection):
      index = collection.normalizedDifference(['B5','B7'])
      reference = index.subtract(index.focal_median(1,'square','pixels'))
      #  return ((image.where(image.gt(0),0).where(image.lt(-1),-1)).multiply(-1)) ;
      quality = reference.where(reference.gt(0),0).where(reference.lt(-1),-1).multiply(-1)
      # quality = reference.qualityMosaic('nd')
      return quality


  # Quality mosaic of the resultant images including the Reference or Base image plus the Analysis image
  BAND_BASE = Reference.map(NBR)
  BAND_ANALYSIS = Analysis.map(NBR)
  type(BAND_BASE)

  QUALITY_ANALYSIS = BAND_ANALYSIS.qualityMosaic('nd')
  QUALITY_BASE = BAND_BASE.qualityMosaic('nd')

  # Defining the Kernel densities and the cleaning region for filtering 
  kernel_size = 0.5;
  min_disturbances = 0.5;
  kernel_clean_size = 0.5;
  threshold_conservative = 0.1;

  # Subtracting the quality mosaic images from each other
  def subtract_(Q_band1, Q_band2):
    rNBR_CHANGE = Q_band2.subtract(Q_band1)
    # rNBR_capped = rNBR_CHANGE.where(rNBR_CHANGE.lt(0), 0)
    return rNBR_CHANGE

  # Calling the subtracted mosaic images
  Difference_NBR = subtract_(QUALITY_BASE , QUALITY_ANALYSIS)

  rNBR_capped = Difference_NBR.where(Difference_NBR.lt(0), 0);
  rNBR_capped.getInfo()
  NBR_difference_capped_1 = Difference_NBR.where(Difference_NBR.lt(threshold_conservative),0) and ((Difference_NBR.where(Difference_NBR.gte(threshold_conservative),1)))

  # cleaning the final NBR output
  NBR_difference_capped_1 = rNBR_capped.where(rNBR_capped.lt(threshold_conservative),0) and ((rNBR_capped.where(rNBR_capped.gte(threshold_conservative),1)));

  # Compute standard deviation (SD) as texture of the NDVI.
  NBR_difference_capped_2 = NBR_difference_capped_1.reduceNeighborhood(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'kernel': ee.Kernel.circle(kernel_clean_size, 'meters'),})

  # Capping the NBR values to capture changes
  NBR_difference_capped_3 = rNBR_capped.where(NBR_difference_capped_2.gte(min_disturbances),1) and ((rNBR_capped.where(NBR_difference_capped_2.lt(min_disturbances),0))).unmask(-2)
  NBR_difference_capped_4 = NBR_difference_capped_3.multiply(rNBR_capped);
  NBR_difference_Export_cleaned = NBR_difference_capped_4.unmask(-2);

  #Masking out the output to dileanate the changes from each other
  masked_changes = NBR_difference_capped_4.updateMask(NBR_difference_capped_4.lte(0));
  masked_changes2 = NBR_difference_capped_4.updateMask(NBR_difference_capped_4.gt(0));

  # Clipping the changes to the region of interest
  clipped_changes = masked_changes2.clip(AOI)

  return clipped_changes





In [ ]:
test = Nbr_product(AOI,Start_date, End_date)

In [ ]:
Map = geemap.Map()
Map.centerObject(AOI)
Map.add_basemap('SATELLITE')
Map.add_layer(test,{'palette': ['FF0000']},'FOREST DISTURBANCE')
# Map.add_layer(AOI,{},'AOI')
Map.addLayerControl()
Map

In [ ]:
def URL_gen(Image,AOI):

    urls = Image.getDownloadUrl({
				'scale': 30, 			
				'crs': 'EPSG:4326', 
				'region': AOI.geometry(),
        'maxPixels': 1e19
			}) 
    return urls


IGA_url = URL_gen(test, AOI)
IGA_url

'https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/thumbnails/6aae6ba7b48c3b1cc4edaec91f095a34-3a6f7ebb25e5303659dae5e15a87e879:getPixels'